In [50]:
import sagemaker
import boto3 ##boto3 is to read and write for aws as pandas is for files(my NASCENT understanding*)
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session 

In [15]:
bucket_name = 'bankappeg'
my_region = boto3.session.Session().region_name
#fetches the region my aws will work better in and i am located in
print(my_region)

us-east-2


In [16]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket = bucket_name,
                         CreateBucketConfiguration
                         ={'LocationConstraint':'us-east-2'})
    print('S3 bucket created')
except Exception as e:
    print('Error: ',e)
    

S3 bucket created


### Nomenclature can be a problem
AWS provides naming standards when naming an aws bucket.

The bucket name can be between 3 and 63 characters long, and can contain only lower-case characters, numbers, periods, and dashes.

Each label in the bucket name must start with a lowercase letter or number.

The bucket name cannot contain underscores, end with a dash, have consecutive periods, or use dashes adjacent to periods.

The bucket name cannot be formatted as an IP address (198.51.100.24).

The name provided contains upper case letters, by switching to lower case letters the issue can be resolved.

In [18]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://bankappeg/xgboost-as-a-built-in-algo/output


### Download data and saving in S3

In [20]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve('https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv','https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv')
    print('Done : downloaded the bank csv')
except Exception as e:
    print('Data loading error', e)
                               

Done : downloaded the bank csv


In [26]:
try:
    datadf = pd.read_csv('./bank_cleaned.csv', index_col=0)
    print('done')
except Exception as e:
    print('error ', e)

done


In [28]:
datadf.head(2)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


## Splitting into train and test

In [ ]:
import numpy as np

train_data, test_data = np.split(datadf.sample(frac=1
                                               ,random_state=0),
                                 [int(0.7 * len(datadf))])
print(train_data.shape,test_data.shape)

In [47]:
# saving train and test in buckets
import os
pd.concat([train_data['y_yes'],
           train_data.drop(['y_no','y_yes'],axis=1)],
          axis=1).to_csv('train.csv',index= False,header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data = 's3://{}/{}/train'.format(bucket_name,prefix),content_type= 'csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [53]:
#'s3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.

In [52]:
#for test data
pd.concat([test_data['y_yes'],test_data.drop(['y_yes','y_no'],axis=1)],axis=1).to_csv('test.csv',index = False,header = False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix),content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


### Build model xgboost

In [54]:
#we use get_image_uri will help us fetch the inbuilt xgboost from a container and put it in a container

container = get_image_uri(boto3.Session().region_name,'xgboost',repo_version='1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [57]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [58]:
estimator = sagemaker.estimator.Estimator(image_name=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [60]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

2020-09-08 22:54:58 Starting - Starting the training job...
2020-09-08 22:55:00 Starting - Launching requested ML instances.........
2020-09-08 22:56:44 Starting - Preparing the instances for training...
2020-09-08 22:57:22 Downloading - Downloading input data
2020-09-08 22:57:22 Training - Downloading the training image...
2020-09-08 22:57:55 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[22:57:51] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?form

## Deploy

In [61]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

### Prediction on test data


In [62]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #we put the data into an array format
xgb_predictor.content_type = 'text/csv' 
xgb_predictor.serializer = csv_serializer
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict
predictions_array = np.fromstring(predictions[1:], sep=',') #prediction into an array
print(predictions_array.shape)

(12357,)


In [63]:
#taken from aws records
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.1%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10830)    33% (147)
Purchase        9% (1079)     67% (301) 



### Deleting endpoints

In [64]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '71B37C10EF451E11',
   'HostId': 'k7xOpFqEu3F99VKhSl/+j9SUz1ayGwVcwIDTBXnIIHDWUGAytoLXI6L6aO4LNBJs3VOBdvg9XTU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'k7xOpFqEu3F99VKhSl/+j9SUz1ayGwVcwIDTBXnIIHDWUGAytoLXI6L6aO4LNBJs3VOBdvg9XTU=',
    'x-amz-request-id': '71B37C10EF451E11',
    'date': 'Wed, 09 Sep 2020 00:16:15 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2020-09-08-22-54-57-600/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'}]}]